<a href="https://colab.research.google.com/github/jjaw89/spring_2025_dl_audio_project/blob/main/MusdbDatasetCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# when you install musdb, pip automatically installs a version of stempeg that
# contains a small bug. To work around this, download the stempeg folder from
# the github to your drive.

%pip install musdb  # has some helpful data structures, also installs ffmpeg and stempeg
%pip uninstall stempeg    # musdb installs the wrong version of stempeg'

# The path below should be changed to the location of the stempeg package in
# your Drive
%cd '/content/drive/MyDrive/DeepLearningBootcamp'

import stempeg
import musdb
import torch
import librosa
import numpy as np
from torch.utils.data import Dataset

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 11.0 MB/s eta 0:00:00
Found existing installation: stempeg 0.2.3
Uninstalling stempeg-0.2.3:
  Would remove:
    /usr/local/bin/stem2files
    /usr/local/lib/python3.11/dist-packages/stempeg-0.2.3.dist-info/*
    /usr/local/lib/python3.11/dist-packages/stempeg/*
Proceed (Y/n)? Y
  Successfully uninstalled stempeg-0.2.3
/content/drive/MyDrive/DeepLearningBootcamp


In [ ]:
############## ONLY RUN THIS CELL IF YOU NEED TO DOWNLOAD DATA #################
import requests

file_url = "https://zenodo.org/records/1117372/files/musdb18.zip"
zip_path = "/content/gdrive/My Drive/Deep Learning Bootcamp/musdb18.zip"
destination_path = "/content/gdrive/My Drive/Deep Learning Bootcamp/musdb18_data"

r = requests.get(file_url, stream = True)
with open(zip_path, "wb") as file:
  for block in r.iter_content(chunk_size = 1024):
    if block:
      file.write(block)

import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/gdrive/My Drive/Deep Learning Bootcamp/musdb18_data")

In [ ]:
class MusdbDataset(Dataset):

  def __init__(self, musDB, window_size = 256, step_size = 128):
    self.mel_specs = torch.zeros(1, 2, 128, window_size)
    self.sample_rates = torch.tensor([0])

    num_songs = 0

    for track in musDB:
      stems, rate = track.stems, track.rate

      num_songs += 1

      # separate the vocal from other instruments and conver to mono signal
      audio_novocal = librosa.to_mono(np.transpose(stems[1] + stems[2] + stems[3]))
      audio_vocal = librosa.to_mono(np.transpose(stems[4]))

      # compute log mel spectrogram and convert to pytorch tensor
      logmelspec_novocal = torch.from_numpy(self._mel_spectrogram(audio_novocal, rate))
      logmelspec_vocal = torch.from_numpy(self._mel_spectrogram(audio_vocal, rate))

      start_ndx = 0

      for step in range(window_size // step_size):
        cropped_logmelspec_novocal = logmelspec_novocal[:, start_ndx:]
        cropped_logmelspec_vocal = logmelspec_vocal[:, start_ndx:]
        num_slices = cropped_logmelspec_novocal.shape[1] // window_size

        # chop off the last bit so that number of stft steps is a multiple of window_size
        cropped_logmelspec_novocal = cropped_logmelspec_novocal[: , 0:num_slices*window_size]
        cropped_logmelspec_vocal = cropped_logmelspec_vocal[:, 0:num_slices*window_size]

        # reshape tensors into chunks of size 128x(window_size)
        # first dimension is number of chunks
        cropped_logmelspec_novocal = torch.transpose(torch.reshape(cropped_logmelspec_novocal, (128, num_slices, window_size)), 0, 1)
        cropped_logmelspec_vocal = torch.transpose(torch.reshape(cropped_logmelspec_vocal, (128, num_slices, window_size)), 0, 1)

        # unsqueeze and concatenate these tensors. Then concatenate to the big tensor
        logmels = torch.cat((cropped_logmelspec_novocal.unsqueeze(1), cropped_logmelspec_vocal.unsqueeze(1)), 1)
        self.mel_specs = torch.cat((self.mel_specs, logmels), 0)
        self.sample_rates = torch.cat((self.sample_rates, torch.full((num_slices,), rate)), 0)

        if num_songs % 5 == 0:
          print(str(num_songs) + " songs processed; produced " + str(self.mel_specs.shape[0]) + " spectrograms")

    # remove the all zeros slice that we initialized with
    self.mel_specs = self.mel_specs[1: , : , : , :]
    self.sample_rates = self.sample_rates[1:]

  def __len__(self):
    return self.mel_specs.shape[0]

  def __getitem__(self, ndx):
    # returns tuple (mel spectrogram of accompaniment, mel spectrogram of vocal, rate)
    return self.mel_specs[ndx, 0], self.mel_specs[ndx, 1], self.sample_rates[ndx]

  def _mel_spectrogram(self, audio, rate):
    # compute the log-mel-spectrogram of the audio at the given sample rate
    return librosa.power_to_db(librosa.feature.melspectrogram(y = audio, sr = rate))

  def cat(self, other_ds):
    self.mel_specs = torch.cat((self.mel_specs, other_ds.mel_specs), 0)
    self.sample_rates = torch.cat((self.sample_rates, other_ds.sample_rates), 0)

In [ ]:
# get the full data set into the workspace
music_train = musdb.DB("/content/drive/MyDrive/DeepLearningBootcamp/musdb18_data", subsets="train")
music_test = musdb.DB("/content/drive/MyDrive/DeepLearningBootcamp/musdb18_data", subsets="train")

In [ ]:
# create a dataset out of all 100 tracks, see how many slices of audio we have
musdbData_train = MusdbDataset(music_train, step_size = 128)
musdbData_test = MusdbDataset(music_test, step_size = 128)

5 songs processed; produced 328 spectrograms
10 songs processed; produced 817 spectrograms
15 songs processed; produced 1182 spectrograms
20 songs processed; produced 1645 spectrograms
25 songs processed; produced 2046 spectrograms
30 songs processed; produced 2473 spectrograms
35 songs processed; produced 2839 spectrograms
40 songs processed; produced 3215 spectrograms
45 songs processed; produced 3649 spectrograms
50 songs processed; produced 4027 spectrograms
55 songs processed; produced 4521 spectrograms
60 songs processed; produced 4602 spectrograms
65 songs processed; produced 4661 spectrograms
70 songs processed; produced 4914 spectrograms
75 songs processed; produced 5392 spectrograms
80 songs processed; produced 5871 spectrograms
85 songs processed; produced 6360 spectrograms
90 songs processed; produced 6752 spectrograms
95 songs processed; produced 7264 spectrograms
100 songs processed; produced 7648 spectrograms


In [ ]:
torch.save(musdbData_train, '/content/drive/MyDrive/DeepLearningBootcamp/musdb_withOverlap_train.pt')
torch.save(musdbData_test, '/content/drive/MyDrive/DeepLearningBootcamp/musdb_withOverlap_test.pt')